In [17]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse

In [18]:
# DataFrame maestro
master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

start_time = time.time()

for url in country_links:
    
    try:
        # Obtener HTML
        response = requests.get(url, headers=headers)
        html = response.text

        # Buscar los datos con regex
        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            # Convertir a DataFrame
            df = pd.DataFrame(data[1:], columns=data[0])  # data[0] = headers
            df.columns = ["Period", "Value"]

            # Extraer nombre del país desde el URL
            country = urlparse(url).path.strip("/").split("/")[0]

            # Agregar columna con el país
            df["Country"] = country

            # Convertir a estructura pivotada
            df_pivot = df.pivot(index="Period", columns="Country", values="Value")

            # Unir con el DataFrame maestro
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    time.sleep(5)  # 💤 Espera de 2 segundos antes de la próxima solicitud

end_time = time.time()
total_time_sleep = len(country_links) * 5
elapsed = end_time - start_time - total_time_sleep

✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
✅ Datos extraídos correctamente para Belgium
✅ Datos extraídos correctamente para Brazil
✅ Datos extraídos correctamente para Bulgaria
✅ Datos extraídos correctamente para Canada
✅ Datos extraídos correctamente para Chile
✅ Datos extraídos correctamente para China
✅ Datos extraídos correctamente para Colombia
✅ Datos extraídos correctamente para Croatia
✅ Datos extraídos correctamente para Cyprus
✅ Datos extraídos correctamente para Czech-Republic
✅ Datos extraídos correctamente para Denmark
✅ Datos extraídos correctamente para Estonia
✅ Datos extraídos correctamente para Finland
✅ Datos extraídos correctamente para France
✅ Datos extraídos correctamente para Germany
✅ Datos extraídos correctamente para Greece
✅ Datos extraídos correctamente para Hong-Kong
✅ Datos extraídos correctamente para Hungary
✅ Datos extraídos correctamente para Iceland
✅ Datos extraídos correctamente para India
✅ Datos 

NameError: name 'total_sleep_time' is not defined

In [19]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

In [20]:
# Separar "Q1 1990" en "Quarter" y "Year"
master_df[["Quarter", "Year"]] = master_df["Period"].str.extract(r"Q(\d)\s+(\d{4})").astype(int)

In [21]:
# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Quarter", "Year"]]
master_df = master_df[cols]

In [22]:
# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [23]:
master_df.head()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
0,1,1990,28.24,51.86,32.75,NaN,NaN,44.35,NaN,NaN,...,NaN,12.41,60.13,29.22,36.53,74.41,NaN,NaN,55.08,34.79
1,2,1990,28.50,58.38,32.67,NaN,NaN,41.38,NaN,NaN,...,NaN,12.98,64.52,29.72,37.27,75.45,NaN,NaN,55.14,34.49
2,3,1990,28.24,64.82,33.63,NaN,NaN,41.60,NaN,NaN,...,NaN,13.28,65.94,30.30,38.00,75.29,NaN,NaN,55.01,35.09
3,4,1990,28.41,65.45,34.01,NaN,NaN,41.93,NaN,NaN,...,NaN,13.55,68.80,30.82,37.45,73.62,NaN,NaN,54.76,34.55
4,1,1991,28.38,67.18,34.21,NaN,NaN,43.96,NaN,NaN,...,NaN,13.97,70.64,32.56,40.02,73.75,51.8,NaN,54.34,34.06


In [24]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
136,1,2024,188.10,208.05,152.14,198.60,195.09,217.64,343.61,131.53,...,165.08,178.09,142.55,113.84,176.17,158.17,175.41,2747.61,228.71,162.59
137,2,2024,192.13,207.82,151.71,202.05,201.81,221.28,351.26,128.21,...,168.71,177.48,142.24,117.89,178.64,159.85,176.04,2914.85,229.86,164.59
138,3,2024,194.75,207.50,154.11,205.52,209.73,217.06,356.74,124.93,...,171.55,177.56,142.34,121.23,179.81,160.71,179.36,3093.81,231.92,168.74
139,4,2024,195.13,207.11,155.29,208.93,215.49,213.42,366.47,122.70,...,176.92,179.25,142.70,123.42,179.34,163.52,181.43,3300.23,234.44,169.58
140,1,2025,NaN,208.84,NaN,212.30,NaN,214.99,NaN,121.64,...,NaN,181.19,142.34,NaN,NaN,164.70,181.54,3622.43,234.42,171.74


In [26]:
print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_time_sleep:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {elapsed:.2f} s")

⏳ Tiempo total (incluyendo esperas): 365.57 s
😴 Tiempo en espera: 285.00 s
⚡ Tiempo efectivo de ejecución: 80.57 s


In [27]:
master_df.size

8319

In [29]:
master_df.shape

(141, 59)

In [31]:
master_df.describe()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
count,141.000000,141.000000,140.000000,141.000000,140.000000,97.000000,108.000000,141.000000,92.000000,80.000000,...,72.000000,141.000000,141.000000,140.000000,140.000000,141.000000,137.000000,61.000000,141.000000,141.000000
mean,2.489362,2007.127660,86.772000,113.646809,85.711786,115.561031,96.803704,103.975319,164.440109,113.548375,...,110.695278,85.173404,94.158794,72.157857,92.679929,100.276879,107.861460,624.179508,112.721135,90.192128
std,1.125148,10.212633,49.806177,44.793182,36.504888,59.307984,45.576140,61.719123,92.590165,20.873835,...,24.780401,57.344328,28.491147,28.439220,52.162268,29.825989,36.771801,926.526738,49.688537,43.449992
min,1.000000,1990.000000,28.240000,51.860000,32.670000,28.550000,32.000000,41.380000,55.920000,75.870000,...,83.420000,12.410000,57.480000,29.220000,31.940000,69.550000,51.800000,97.390000,54.340000,31.770000
25%,1.000000,1998.000000,36.242500,81.240000,49.315000,48.750000,63.277500,47.200000,83.370000,99.447500,...,94.620000,25.240000,65.890000,39.015000,40.802500,73.770000,77.270000,139.770000,67.140000,43.480000
50%,2.000000,2007.000000,82.620000,89.330000,94.465000,140.820000,95.580000,90.430000,136.185000,111.625000,...,101.695000,95.450000,93.840000,76.050000,87.665000,90.770000,94.420000,220.410000,103.910000,97.760000
75%,3.000000,2016.000000,127.010000,141.660000,110.747500,156.760000,123.875000,148.430000,233.707500,134.170000,...,115.680000,137.540000,115.770000,95.687500,143.275000,124.190000,142.820000,389.640000,138.460000,123.690000
max,4.000000,2025.000000,195.130000,218.660000,155.290000,212.300000,215.490000,252.680000,366.470000,145.910000,...,176.920000,181.190000,154.120000,123.420000,193.140000,164.700000,181.540000,3622.430000,234.440000,171.740000


In [32]:
master_df.dtypes

Quarter             int64
Year                int64
Australia         float64
Austria           float64
Belgium           float64
Brazil            float64
Bulgaria          float64
Canada            float64
Chile             float64
China             float64
Colombia          float64
Croatia           float64
Cyprus            float64
Czech-Republic    float64
Denmark           float64
Estonia           float64
Finland           float64
France            float64
Germany           float64
Greece            float64
Hong-Kong         float64
Hungary           float64
Iceland           float64
India             float64
Indonesia         float64
Ireland           float64
Israel            float64
Italy             float64
Japan             float64
Latvia            float64
Lithuania         float64
Luxembourg        float64
Macedonia         float64
Malaysia          float64
Malta             float64
Mexico            float64
Morocco           float64
Netherlands       float64
New-Zealand 

In [33]:
master_df.to_parquet("hpi_countries.parquet", index=False)